# Week 8: Multi-Agent Deal Hunting System

This notebook demonstrates Week 8 concepts:
- Multi-agent architecture with specialized agents
- Real-time Gradio UI with threading and queues
- Integration with deployed Modal services
- RAG pipeline with ChromaDB
- Ensemble model combining multiple AI approaches

In [ ]:
!pip install -q gradio pydantic openai chromadb sentence-transformers scikit-learn feedparser beautifulsoup4 requests plotly

In [ ]:
import os
import logging
import queue
import threading
import time
import json
from typing import List, Optional
from datetime import datetime

import gradio as gr
import plotly.graph_objects as go
from pydantic import BaseModel

In [ ]:
class Deal(BaseModel):
    product_description: str
    price: float
    url: str


class DealSelection(BaseModel):
    deals: List[Deal]


class Opportunity(BaseModel):
    deal: Deal
    estimate: float
    discount: float


class MockAgent:
    name = "Mock Agent"
    color = '\033[37m'
    
    def log(self, message):
        logging.info(f"[{self.name}] {message}")


class MockScannerAgent(MockAgent):
    name = "Scanner Agent"
    
    def scan(self, memory=None):
        self.log("Simulating RSS feed scan")
        time.sleep(1)
        
        deals = [
            Deal(
                product_description="Apple iPad Pro 11-inch 256GB WiFi (latest model) - Space Gray. Features M2 chip, Liquid Retina display, 12MP camera, Face ID, and all-day battery life.",
                price=749.99,
                url="https://example.com/ipad"
            ),
            Deal(
                product_description="Sony WH-1000XM5 Wireless Noise Cancelling Headphones - Industry-leading noise cancellation, exceptional sound quality, 30-hour battery life, comfortable design.",
                price=329.99,
                url="https://example.com/sony-headphones"
            )
        ]
        
        return DealSelection(deals=deals)


class MockEnsembleAgent(MockAgent):
    name = "Ensemble Agent"
    
    def price(self, description: str) -> float:
        self.log(f"Estimating price for product")
        time.sleep(0.5)
        
        if "iPad" in description:
            return 899.00
        elif "Sony" in description:
            return 398.00
        else:
            return 150.00


class MockMessagingAgent(MockAgent):
    name = "Messaging Agent"
    
    def alert(self, opportunity: Opportunity):
        self.log(f"Alert sent: ${opportunity.discount:.2f} discount on {opportunity.deal.product_description[:50]}...")


class MockPlanningAgent(MockAgent):
    name = "Planning Agent"
    DEAL_THRESHOLD = 50
    
    def __init__(self):
        self.scanner = MockScannerAgent()
        self.ensemble = MockEnsembleAgent()
        self.messenger = MockMessagingAgent()
    
    def plan(self, memory=None) -> Optional[Opportunity]:
        if memory is None:
            memory = []
        
        self.log("Starting planning cycle")
        
        selection = self.scanner.scan(memory)
        
        if selection and selection.deals:
            opportunities = []
            for deal in selection.deals:
                estimate = self.ensemble.price(deal.product_description)
                discount = estimate - deal.price
                opportunities.append(Opportunity(
                    deal=deal,
                    estimate=estimate,
                    discount=discount
                ))
            
            opportunities.sort(key=lambda x: x.discount, reverse=True)
            best = opportunities[0]
            
            self.log(f"Best deal has discount: ${best.discount:.2f}")
            
            if best.discount > self.DEAL_THRESHOLD:
                self.messenger.alert(best)
                return best
        
        return None


class MockDealAgentFramework:
    MEMORY_FILE = "mock_memory.json"
    
    def __init__(self):
        self.memory = self.read_memory()
        self.planner = None
    
    def init_agents_as_needed(self):
        if not self.planner:
            logging.info("Initializing Mock Agent Framework")
            self.planner = MockPlanningAgent()
            logging.info("Mock Agent Framework ready")
    
    def read_memory(self) -> List[Opportunity]:
        if os.path.exists(self.MEMORY_FILE):
            try:
                with open(self.MEMORY_FILE, 'r') as f:
                    data = json.load(f)
                return [Opportunity(**item) for item in data]
            except:
                return []
        return []
    
    def write_memory(self):
        data = [opp.dict() for opp in self.memory]
        with open(self.MEMORY_FILE, 'w') as f:
            json.dump(data, f, indent=2)
    
    def run(self) -> List[Opportunity]:
        self.init_agents_as_needed()
        result = self.planner.plan(memory=self.memory)
        
        if result:
            self.memory.append(result)
            self.write_memory()
        
        return self.memory
    
    @classmethod
    def get_plot_data(cls, max_datapoints=100):
        import numpy as np
        
        n_points = min(100, max_datapoints)
        vectors = np.random.randn(n_points, 3)
        documents = [f"Product {i}" for i in range(n_points)]
        colors = ['red', 'blue', 'green', 'orange'] * (n_points // 4 + 1)
        
        return documents[:n_points], vectors, colors[:n_points]

In [ ]:
BG_BLACK = '\033[40m'
RED = '\033[31m'
GREEN = '\033[32m'
YELLOW = '\033[33m'
BLUE = '\033[34m'
MAGENTA = '\033[35m'
CYAN = '\033[36m'
WHITE = '\033[37m'
BG_BLUE = '\033[44m'
RESET = '\033[0m'

color_mapper = {
    BG_BLACK+RED: "#dd0000",
    BG_BLACK+GREEN: "#00dd00",
    BG_BLACK+YELLOW: "#dddd00",
    BG_BLACK+BLUE: "#0000ee",
    BG_BLACK+MAGENTA: "#aa00dd",
    BG_BLACK+CYAN: "#00dddd",
    BG_BLACK+WHITE: "#87CEEB",
    BG_BLUE+WHITE: "#ff7800"
}


def reformat_log(message):
    for key, value in color_mapper.items():
        message = message.replace(key, f'<span style="color: {value}">')
    message = message.replace(RESET, '</span>')
    return message


class QueueHandler(logging.Handler):
    def __init__(self, log_queue):
        super().__init__()
        self.log_queue = log_queue

    def emit(self, record):
        self.log_queue.put(self.format(record))


def setup_logging(log_queue):
    handler = QueueHandler(log_queue)
    formatter = logging.Formatter(
        "[%(asctime)s] %(message)s",
        datefmt="%Y-%m-%d %H:%M:%S",
    )
    handler.setFormatter(formatter)
    logger = logging.getLogger()
    logger.addHandler(handler)
    logger.setLevel(logging.INFO)


def html_for(log_data):
    output = '<br>'.join(log_data[-20:])
    return f"""
    <div style="height: 420px; overflow-y: auto; border: 1px solid #444; background-color: #1a1a1a; padding: 12px; font-family: monospace; font-size: 13px; color: #fff;">
        {output}
    </div>
    """


def get_plot():
    try:
        documents, vectors, colors = MockDealAgentFramework.get_plot_data(max_datapoints=100)
        
        fig = go.Figure(data=[go.Scatter3d(
            x=vectors[:, 0],
            y=vectors[:, 1],
            z=vectors[:, 2],
            mode='markers',
            marker=dict(size=3, color=colors, opacity=0.7),
        )])
        
        fig.update_layout(
            scene=dict(
                xaxis_title='X', 
                yaxis_title='Y', 
                zaxis_title='Z',
                aspectmode='manual',
                aspectratio=dict(x=2.2, y=2.2, z=1),
                camera=dict(eye=dict(x=1.6, y=1.6, z=0.8)),
                bgcolor='#1a1a1a'
            ),
            height=420,
            margin=dict(r=5, b=5, l=5, t=5),
            paper_bgcolor='#1a1a1a',
            font=dict(color='#ffffff'),
            title="Mock Vector Space (Random Data for Demo)"
        )
        return fig
        
    except Exception as e:
        fig = go.Figure()
        fig.update_layout(
            title=f'Error: {str(e)}',
            height=420,
            paper_bgcolor='#1a1a1a',
            font=dict(color='#ffffff')
        )
        return fig

In [ ]:
class DealHunterApp:
    
    def __init__(self):
        self.framework = None
    
    def get_framework(self):
        if not self.framework:
            self.framework = MockDealAgentFramework()
            self.framework.init_agents_as_needed()
        return self.framework
    
    def opportunities_to_table(self, opportunities):
        if not opportunities:
            return []
        
        return [
            [
                opp.deal.product_description,
                f"${opp.deal.price:.2f}",
                f"${opp.estimate:.2f}",
                f"${opp.discount:.2f}",
                opp.deal.url
            ]
            for opp in opportunities
            if isinstance(opp, Opportunity)
        ]
    
    def scan_for_deals(self):
        framework = self.get_framework()
        logging.info(f"Scan triggered at {datetime.now().strftime('%H:%M:%S')} - Current memory: {len(framework.memory)} deals")
        new_opportunities = framework.run()
        logging.info(f"Scan complete - Total deals: {len(framework.memory)}")
        return self.opportunities_to_table(new_opportunities)
    
    def scan_with_logging(self, log_data):
        log_queue = queue.Queue()
        result_queue = queue.Queue()
        setup_logging(log_queue)
        
        def worker():
            result = self.scan_for_deals()
            result_queue.put(result)
        
        thread = threading.Thread(target=worker)
        thread.start()
        
        framework = self.get_framework()
        current_table = self.opportunities_to_table(framework.memory)
        
        while True:
            try:
                message = log_queue.get_nowait()
                log_data.append(reformat_log(message))
                current_table = self.opportunities_to_table(framework.memory)
                yield log_data, html_for(log_data), current_table
            except queue.Empty:
                try:
                    final_table = result_queue.get_nowait()
                    yield log_data, html_for(log_data), final_table
                    return
                except queue.Empty:
                    current_table = self.opportunities_to_table(framework.memory)
                    yield log_data, html_for(log_data), current_table
                    time.sleep(0.1)
    
    def handle_selection(self, selected_index: gr.SelectData):
        framework = self.get_framework()
        row = selected_index.index[0]
        
        if row < len(framework.memory):
            opportunity = framework.memory[row]
            framework.planner.messenger.alert(opportunity)
            return f"Alert sent for: {opportunity.deal.product_description[:60]}..."
        
        return "Invalid selection"
    
    def load_initial_state(self):
        framework = self.get_framework()
        initial_table = self.opportunities_to_table(framework.memory)
        return [], "", initial_table
    
    def launch(self):
        with gr.Blocks(title="The Price is Right", fill_width=True) as ui:
            
            log_data = gr.State([])
            
            gr.Markdown(
                '<div style="text-align: center; font-size: 28px; font-weight: bold; margin: 20px 0;">The Price is Right - Demo</div>'
                '<div style="text-align: center; font-size: 16px; color: #666; margin-bottom: 20px;">Multi-Agent Deal Hunting System (Mock Version)</div>'
            )
            
            with gr.Row():
                opportunities_table = gr.Dataframe(
                    headers=["Product Description", "Price", "Estimate", "Discount", "URL"],
                    wrap=True,
                    column_widths=[6, 1, 1, 1, 3],
                    row_count=10,
                    col_count=5,
                    max_height=420,
                    interactive=False
                )
            
            with gr.Row():
                with gr.Column(scale=1):
                    logs_display = gr.HTML(label="Agent Logs")
                with gr.Column(scale=1):
                    vector_plot = gr.Plot(value=get_plot(), show_label=False)
            
            ui.load(
                self.load_initial_state,
                inputs=[],
                outputs=[log_data, logs_display, opportunities_table]
            )
            
            timer = gr.Timer(value=10, active=True)
            timer.tick(
                self.scan_with_logging,
                inputs=[log_data],
                outputs=[log_data, logs_display, opportunities_table]
            )
            
            selection_feedback = gr.Textbox(visible=False)
            opportunities_table.select(
                self.handle_selection,
                inputs=[],
                outputs=[selection_feedback]
            )
        
        ui.launch(share=True, inbrowser=True)

In [ ]:
app = DealHunterApp()
app.launch()